In [1]:
import re
import sys
from time import time, sleep
from pprint import pprint
from functools import partial
from multiprocessing import Pool

import numpy as np
import pandas as pd
import optuna
import matplotlib.pyplot as plt

FILENAME = "main.cpp"

In [2]:
!g++ {FILENAME} -std=gnu++17 -Wall -Wextra -O2 -DONLINE_JUDGE -o {FILENAME}.out

main.cpp:49: warning: ignoring #pragma clang attribute [-Wunknown-pragmas]
   49 | #pragma clang attribute push (__attribute__((target("arch=skylake"))),apply_to=function)
      | 
main.cpp:1845: warning: ignoring #pragma clang attribute [-Wunknown-pragmas]
 1845 | #pragma clang attribute pop
      | 
main.cpp: In function ‘void test::TestBitBoardGetSpeed()’:
main.cpp:854:9: warning: unused variable ‘t0’ [-Wunused-variable]
  854 |  double t0;
      |         ^~
main.cpp: In function ‘void globals::UpdateValueTable()’:
main.cpp:961:15: warning: unused variable ‘vm’ [-Wunused-variable]
  961 |   const auto& vm = v_modified[idx_RCSEV];
      |               ^~
main.cpp: In member function ‘void State::RecalcFutureScore()’:
main.cpp:1067:14: warning: unused variable ‘old_subscore2’ [-Wunused-variable]
 1067 |   const auto old_subscore2 = subscore2;
      |              ^~~~~~~~~~~~~
main.cpp: In function ‘void Solve()’:
main.cpp:1652:54: warning: suggest parentheses around ‘-’ in operand 

In [3]:
N_TESTCASES = 500
TESTCASE_DIRECTORY = "../exp013_optim/in/"
TESTCASES = [f"{i:04d}.txt" for i in range(N_TESTCASES)]

In [4]:
# 速度チェック
# 並列実行すると半分くらいの速度に

!mkdir out

def parallel(i):
    testcase = TESTCASES[i]
    i = f"{i:04d}"
    t0 = time()
    !./{FILENAME}.out < {TESTCASE_DIRECTORY}{testcase} > ./out/{testcase}
    t = time() - t0
    score = !python3 ../A/tester/judge.py {TESTCASE_DIRECTORY}{testcase} ./out/{testcase}
    print(i, f"time:{t:.4f}", *score)
    score = int(score[0].split(":")[-1])
    return score, t

parallel(0)

mkdir: cannot create directory ‘out’: File exists
elapsed_time=0.00894475  cum_base_sec/elapsed_time=6.9221e+09  modified=6.34886e+09
remaining_time=1.60706
turn=49
time / width = 2.06372e-06
beam_width=130
q.size()=296

elapsed_time=0.0182744  cum_base_sec/elapsed_time=8.78762e+09  modified=7.84536e+09
remaining_time=1.59773
turn=99
time / width = 1.42295e-06
beam_width=189
q.size()=391

elapsed_time=0.0326018  cum_base_sec/elapsed_time=9.2642e+09  modified=8.55913e+09
remaining_time=1.58341
turn=149
time / width = 1.54684e-06
beam_width=235
q.size()=499

elapsed_time=0.0515914  cum_base_sec/elapsed_time=9.39189e+09  modified=8.89391e+09
remaining_time=1.56442
turn=199
time / width = 1.74985e-06
beam_width=271
q.size()=502

elapsed_time=0.075152  cum_base_sec/elapsed_time=9.40357e+09  modified=9.04542e+09
remaining_time=1.54086
turn=249
time / width = 1.86113e-06
beam_width=301
q.size()=512

elapsed_time=0.104599  cum_base_sec/elapsed_time=9.2359e+09  modified=8.98337e+09
remaining_ti

(5211697, 1.897125005722046)

In [ ]:
# もとのスコアの確認

scores = []
times = []

!mkdir out

def parallel(i):
    testcase = TESTCASES[i]
    i = f"{i:04d}"
    t0 = time()
    if i == "0123":
        !./{FILENAME}.out < {TESTCASE_DIRECTORY}{testcase} > ./out/{testcase}
    else:
        !./{FILENAME}.out < {TESTCASE_DIRECTORY}{testcase} > ./out/{testcase} 2> /dev/null
    t = time() - t0
    score = !python3 ../A/tester/judge.py {TESTCASE_DIRECTORY}{testcase} ./out/{testcase}
    print(i, f"time:{t:.4f}", *score)
    score = int(score[0].split(":")[-1])
    return score, t

for i in range(N_TESTCASES):
    score, t = parallel(i)
    scores.append(score)
    times.append(t)

# with Pool(30) as p:
#     for score, t in p.map(parallel, range(N_TESTCASES)):
#         scores.append(score)
#         times.append(t)

np.mean(scores) * 50 / 1e6, np.std(scores) * 50 / 1e6

mkdir: cannot create directory ‘out’: File exists
0000 time:1.9094 score:5349510
0001 time:1.9011 score:5278297
0002 time:1.8870 score:5235426
0003 time:1.8900 score:4785993
0004 time:1.9094 score:5529853
0005 time:1.8960 score:4997468
0006 time:1.8640 score:4855300
0007 time:1.8941 score:5274382
0008 time:1.8766 score:4303674
0009 time:1.8861 score:5446058
0010 time:1.8768 score:5146119
0011 time:1.8901 score:5599367
0012 time:1.8391 score:4228561
0013 time:1.8782 score:5053176
0014 time:1.8859 score:4837054
0015 time:1.8643 score:4902735
0016 time:1.8935 score:5263087
0017 time:1.9421 score:5690887
0018 time:1.8497 score:5363361
0019 time:1.9014 score:5356796
0020 time:1.8877 score:5505467
0021 time:1.8748 score:5143936
0022 time:1.8972 score:4886358
0023 time:1.8687 score:4614120
0024 time:1.8828 score:5204203
0025 time:1.8950 score:4800452
0026 time:1.8799 score:5188965
0027 time:1.8984 score:5015335
0028 time:1.9166 score:5335388
0029 time:1.8924 score:5517348
0030 time:1.8972 sco

0152 time:1.8890 score:5400768
0153 time:1.8838 score:5302249
0154 time:1.8978 score:5851054
0155 time:1.8911 score:5340284
0156 time:1.8733 score:4728744
0157 time:1.8864 score:4609533
0158 time:1.9002 score:5476536
0159 time:1.9078 score:5280593
0160 time:1.9007 score:4787278
0161 time:1.8674 score:5169833
0162 time:1.9079 score:4785273
0163 time:1.8705 score:5122620
0164 time:1.8952 score:5358180
0165 time:1.8796 score:5308475
0166 time:1.9007 score:4843052
0167 time:1.8808 score:5241325
0168 time:1.8453 score:5167288
0169 time:1.8527 score:4880093
0170 time:1.8656 score:5237622
0171 time:1.8756 score:5118039
0172 time:1.8538 score:5234810
0173 time:1.8825 score:5677316
0174 time:1.9008 score:4556137
0175 time:1.8895 score:4907432
0176 time:1.9042 score:4904949
0177 time:1.8964 score:5336252
0178 time:1.8938 score:5234785
0179 time:1.8699 score:5193497
0180 time:1.8818 score:4995256
0181 time:1.8848 score:4980814
0182 time:1.8838 score:5565360
0183 time:1.8846 score:4531987
0184 tim